<a href="https://colab.research.google.com/github/Rashi2011/Rashi-Madhukar/blob/master/Healthcare%20Project/N_Layer_Model_on_Heart_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics


In [2]:
#Loading the data()
#Downloading the dataset from local drive
from google.colab import files
uploaded = files.upload()


Saving Heart.csv to Heart.csv


In [7]:
import io
dataset = pd.read_csv(io.BytesIO(uploaded['Heart.csv']))

In [29]:
dataset = dataset.dropna()
X = dataset.iloc[:,1:14].values
Y = dataset.iloc[:,14].values

#Encoding Categorical Data
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
lb = LabelEncoder()
X[:,2] = lb.fit_transform(X[:,2])       #chest pain(typical,asymptomatic,non_typical,non-anginal)
f=X[:,12].tolist()

for i in range(len(f)):
    if f[i]=='normal':
        f[i]=3
    if f[i]=='fixed':
        f[i]=6
    if f[i]=='reversable':
        f[i]=7

X[:,12]=f
#use one hot encoder for chest pain column
ct = ColumnTransformer([("Chest Pain", OneHotEncoder(), [2])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:,1:]


#imputer(It takes 2d array)
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
X = imp.fit_transform(X)
Y = lb.fit_transform(Y)

#Plotting the distribution of class labels
count_p=0;count_n=0
for i in Y:
    if i==0:
        count_n+=1
    else:
        count_p+=1
print("No: of positive Labels : ", count_p," and No: of -ve labels : ",count_n)

#OneHotEncoder
'''
ohe = OneHotEncoder(categorical_features = [3,13])
X = ohe.fit_transform(X).toarray()
'''

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
print(X.shape)


No: of positive Labels :  137  and No: of -ve labels :  160
(297, 15)


In [30]:
#Splitting train and test set
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.3,random_state = 0)

#Reshaping X and Y(I/p and O/p)
X_train = np.transpose(X_train)
X_test = np.transpose(X_test)
Y_train= np.expand_dims(Y_train, axis=0)
Y_test= np.expand_dims(Y_test, axis=0)
print(X_train.shape)
print(Y_train.shape)

(15, 207)
(1, 207)


In [66]:
#Initialize Parameters
def initialize_param(layers_dims):
  parameters = {}                #empty dictionary
  L = len(layers_dims)           # no of layers in the network
  for l in range(1,L):
    parameters["W" + str(l)]= np.round(np.random.rand(layers_dims[l],layers_dims[l-1]),4)
    parameters["b"+ str(l)]= np.round(np.zeros((layers_dims[l],1)),3)

  return parameters

In [67]:
'''
layers_dims = [14,5,3,1]
parameters = initialize_param(layers_dims)
#print(parameters)             #Dictionary
print("W1 = "+ str(parameters["W1"]))
print("b1 = "+ str(parameters["b1"]))
print("W2 = "+ str(parameters["W2"]))
print("b2 = "+ str(parameters["b2"]))
'''

'\nlayers_dims = [14,5,3,1]\nparameters = initialize_param(layers_dims)\n#print(parameters)             #Dictionary\nprint("W1 = "+ str(parameters["W1"]))\nprint("b1 = "+ str(parameters["b1"]))\nprint("W2 = "+ str(parameters["W2"]))\nprint("b2 = "+ str(parameters["b2"]))\n'

In [68]:
#Activation Function
def sigmoid(z):
  A = 1/(1+np.exp(-z))
  return A

def relu(z):
  A = np.maximum(0,z)
  return A

def sign_f(AL,mean):
    if AL <= mean:
      return 0
    else:
      return 1


In [69]:
#forward
def forward(A, W, b):
  Z = np.dot(W,A) +b
  #print("Z.shape = ",Z.shape)       
  
  return Z

In [70]:
#Activations forward
def activation_forward(A_prev, W, b ,activation):
  if activation == "sigmoid":
    #print("W,b",W.shape,b.shape)
    Z = forward(A_prev,W,b)
    A  = sigmoid(Z)
    

  elif activation == "relu":
    #print("W,b",W.shape,b.shape)
    Z = forward(A_prev,W,b)
    A  =  relu(Z)
    #print("Ashapes",A.shape)
          
  return A,Z

In [71]:
#Forward Propagation
def forward_model(X,parameters):    
  cache = {}  #empty dictionary    
  A = X
  L = len(parameters)//2
 
  for l in range(1,L):
    A_prev =A
    A,cache["Z"+str(l)] = activation_forward(A_prev, parameters['W'+ str(l)], parameters['b'+ str(l)], activation = "relu")  #i = 1,i = 2
    cache["A"+str(l)] = A
    
    
  AL,cache["Z"+str(L)] = activation_forward(cache["A"+str(L-1)], parameters["W"+str(L)], parameters["b"+str(L)], activation = "sigmoid")  #i = 3
  cache["A"+str(L)] = AL
  return AL,cache



In [72]:
#tocheck'''
'''
AL,cache = forward_model(X_train,parameters)
print(AL.shape)
'''

'\nAL,cache = forward_model(X_train,parameters)\nprint(AL.shape)\n'

In [73]:
#Cost Function
def compute_cost(AL,Y):
  m = Y.shape[1]
  #print(m)
  cost = -(1/m)*np.sum((Y*np.log(AL) + (1-Y)*np.log(1-AL)))
  return cost

In [74]:
#to check
'''
cost = compute_cost(AL,Y_train)
print("cost = ",cost)
'''

'\ncost = compute_cost(AL,Y_train)\nprint("cost = ",cost)\n'

In [75]:

def sigmoid_backward(dA,AL):
  
  dZ = np.multiply(AL,(1-AL))
  Loss = np.multiply(dA,AL)             #(1x207)
  return Loss


def relu_backward(dA,AL):
  dZ= np.zeros((AL.shape))
  for i in range(AL.shape[0]):
    for j in range(AL.shape[1]):
        
        if AL[i][j] <= 0:
          dZ[i][j] = dZ[i][j]+ 0
        else:
          dZ[i][j] =dZ[i][j] + 1
  #print(dZ.shape)               #(5x207)
  dZ = np.multiply(dZ,AL)
  Loss = np.multiply(dA,dZ)

  return Loss

In [76]:
#Backward 
def backward(dA,W,A_prev,Loss): 
  m = dA.shape[1]            
  dW = (1/m)*np.dot(Loss,A_prev.T)   #(1x3)
  dA_ = np.dot(W.T,dA)       #3x207
  #print("dA_",dA_.shape)
  db = (1/m)*np.sum(Loss,axis = 1,keepdims = True) 

  return dA_,dW,db

In [77]:
#Activation backward
def activation_backward(dA,W,A_prev,AL ,activation):
  
  if activation == "relu":
    Loss = relu_backward(dA, AL )
    dA_,dW ,db = backward(dA,W,A_prev,Loss)
  elif activation == "sigmoid":
    Loss = sigmoid_backward(dA,AL)
    dA_,dW ,db = backward(dA,W,A_prev,Loss)

  return dA_,dW,db



In [78]:
#backward Model
def backward_model(AL,X,Y, cache,parameters):
  
  grads = {}           #empty dictionary
  L = int(len(parameters)/2)              #3    #no of layers  
  #print(L)  #no of layers
  m = Y.shape[1]                #no of training examples
  Y = Y.reshape(AL.shape)       #to ensure shape of Y = AL
  cache["A" + str(0)] = X_train
  #intialization of backward propagation
  dAL = AL- Y           #dA3
  #print("dAL",dAL.shape)
  #Sigmoid (for layer L = L-1 )       #L = 3
  
  grads["dA"+ str(L-1)],grads["dW"+ str(L)] ,grads["db"+ str(L)]  = activation_backward(dAL,parameters["W"+str(L)],cache["A" + str(L-1)],cache["A"+ str(L)] ,activation = "sigmoid")

  # for layer (L =L-2 to  L=0 ) 
  for l in reversed(range(L-1)):    #L-1 = 2, #(0 to 2) # means l = 0,1 # reversing it  l = 1,0
    if l!=0:
      grads["dA"+ str(l)],grads["dW"+ str(l+1)] ,grads["db"+ str(l+1)]  = activation_backward(grads["dA"+ str(l+1)], parameters["W"+ str(l+1)],cache["A"+str(l)],cache["A"+str(l+1)] ,activation = "relu")

    else:
      _,grads["dW"+ str(l+1)] ,grads["db"+ str(l+1)]  = activation_backward(grads["dA"+ str(l+1)], parameters["W"+ str(l+1)],cache["A"+ str(l)] ,cache["A"+str(l+1)],activation = "relu")
  return grads
     




In [79]:
#To check
'''
grads = backward_model(AL,X_train,Y_train,cache,parameters)
#print(grads)
'''

'\ngrads = backward_model(AL,X_train,Y_train,cache,parameters)\n#print(grads)\n'

In [80]:
#Update Parameters
def update_param(parameters,grads,learning_rate):
  L = len(parameters)//2
  for l in range(L):
    parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - (learning_rate*(grads["dW"+ str(l+1)]))
    parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - (learning_rate*(grads["db"+ str(l+1)]))

  return parameters

In [81]:

#To Check
'''
parameters = update_param(parameters,grads,learning_rate = 0.01)

print(parameters["W"+str(1)])
print(parameters["b"+str(1)])
'''

'\nparameters = update_param(parameters,grads,learning_rate = 0.01)\n\nprint(parameters["W"+str(1)])\nprint(parameters["b"+str(1)])\n'

In [82]:
from sklearn.metrics import precision_score
#NN Model
def L_layer_model(X, Y, layers_dims, learning_rate , num_iterations):
  costs = []                #empty list
  acc = []
  
  parameters = initialize_param(layers_dims)
  for i in range(0,num_iterations):
    #Forward Propagation(relu to sigmoid)
    
    AL,cache = forward_model(X,parameters)
    

    #compute cost
    cost = compute_cost(AL,Y)

    #Backward Propagation
    grads = backward_model(AL,X,Y, cache,parameters)

    #Update Parameters
    parameters = update_param(parameters,grads,learning_rate)
    

    #Print the cost on every 100th training examples
    
    print ("Cost after iteration %i: %f" %(i, cost))
    
    #Converting to y_pred
    
    y_pred = []
    for j in range(AL.shape[1]):
      y_pred.append(sign_f(AL[0][j],np.mean(AL)))
    y_pred = np.asmatrix(y_pred)
    
    costs.append(cost)
    #Accuracy
    print("acc = {:.2f}".format(metrics.f1_score(Y, y_pred,average = 'micro')))
    print("Precison_Score   : {:.2f}%".format(precision_score(Y_test,y,average = 'micro')*100))
    acc.append(acc)
         
  
  return parameters,costs,acc



In [83]:
layers_dims = [15,7,5,1]
parameters,costs,acc = L_layer_model(X_train, Y_train, layers_dims, 0.1,num_iterations = 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """


Cost after iteration 0: nan
acc = 0.68
Precison_Score   : 87.50%
Cost after iteration 1: 1.231280
acc = 0.66
Precison_Score   : 87.50%
Cost after iteration 2: 0.785098
acc = 0.64
Precison_Score   : 87.50%
Cost after iteration 3: 0.662268
acc = 0.61
Precison_Score   : 87.50%
Cost after iteration 4: 0.652337
acc = 0.62
Precison_Score   : 87.50%
Cost after iteration 5: 0.643672
acc = 0.61
Precison_Score   : 87.50%
Cost after iteration 6: 0.635501
acc = 0.63
Precison_Score   : 87.50%
Cost after iteration 7: 0.627925
acc = 0.64
Precison_Score   : 87.50%
Cost after iteration 8: 0.620875
acc = 0.65
Precison_Score   : 87.50%
Cost after iteration 9: 0.614107
acc = 0.65
Precison_Score   : 87.50%
Cost after iteration 10: 0.607761
acc = 0.65
Precison_Score   : 87.50%
Cost after iteration 11: 0.601533
acc = 0.67
Precison_Score   : 87.50%
Cost after iteration 12: 0.595651
acc = 0.66
Precison_Score   : 87.50%
Cost after iteration 13: 0.589900
acc = 0.67
Precison_Score   : 87.50%
Cost after iteration 

In [85]:
#Prediction
print(X_test.shape,Y_test.shape)
AL,_ = forward_model(X_test,parameters)
cost = compute_cost(AL,Y_test)

y = np.zeros(AL.shape)
print(AL)
print(np.mean(AL))
print(type(y),AL.shape[1])
for i in range(0,AL.shape[1]):
  y[0][i]  = y[0][i] + sign_f(AL[0][i],np.mean(AL)) 
print(y)


print("cost = {:.3f}, acc = {:.3f}".format(cost,metrics.f1_score(Y_test, y,average = 'micro')))
print("Precison_Score   : {:.2f}%".format(precision_score(Y_test,y,average = 'micro')*100))

(15, 90) (1, 90)
[[0.22294335 0.38391126 0.67005422 0.38391126 0.29780392 0.38391126
  0.70069277 0.30879993 0.96963972 0.38391126 0.57389547 0.25808238
  0.9284523  0.29878071 0.40633505 0.34602916 0.99026602 0.38391126
  0.90865185 0.91598984 0.38391126 0.97514529 0.38391126 0.10992479
  0.3383586  0.38391126 0.33279789 0.20698787 0.38391126 0.25615749
  0.90970368 0.38391126 0.99037688 0.96919894 0.38391126 0.99569433
  0.37689066 0.38391126 0.99895342 0.37902926 0.32097818 0.89603296
  0.29462386 0.3675159  0.38391126 0.62158807 0.7562321  0.38391126
  0.38391126 0.38391126 0.99172403 0.99783688 0.94574368 0.38391126
  0.28938267 0.99205528 0.38391126 0.56406465 0.38587499 0.38391126
  0.99856201 0.38391126 0.38391126 0.38391126 0.38391126 0.30548859
  0.38391126 0.97554449 0.38391126 0.35546244 0.15911222 0.38391126
  0.35441991 0.97758775 0.38391126 0.37195492 0.9999159  0.38391126
  0.96817441 0.52040483 0.38391126 0.97695176 0.99531202 0.38391126
  0.38391126 0.38391126 0.36692